In [1]:
import time
import csv
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from urllib.parse import urljoin

# 셀레니움 설정 (창을 띄운 상태로 실행)
options = Options()
# options.add_argument("--headless")  # 필요 시 활성화
driver = webdriver.Chrome(options=options)

def get_weapon_articles_from_page(page_url):
    driver.get(page_url)
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, "html.parser")

    articles_data = []
    article_blocks = soup.select("h4.titles > a")

    for article in article_blocks:
        title = article.text.strip()
        link = urljoin(page_url, article['href'])

        driver.get(link)
        time.sleep(1)
        detail_soup = BeautifulSoup(driver.page_source, "html.parser")
        content_div = detail_soup.select_one("article#article-view-content-div")

        if content_div:
            paragraphs = content_div.find_all("p")
            content = "\n".join([p.text.strip() for p in paragraphs])
        else:
            content = ""

        articles_data.append({
            "기사제목": title,
            "기사내용": content
        })

    return articles_data

def crawl_weapon_articles(base_url, output_csv):
    all_articles = []

    for page in range(1, 21):  # 1~20페이지
        page_url = f"{base_url}?page={page}&sc_section_code=S1N5&view_type=sm"
        print(f"크롤링 중: {page} 페이지")
        articles = get_weapon_articles_from_page(page_url)

        if not articles:
            print(f"{page} 페이지에 기사 없음. 종료합니다.")
            break

        all_articles.extend(articles)

    with open(output_csv, mode="w", newline="", encoding="utf-8-sig") as f:
        writer = csv.DictWriter(f, fieldnames=["기사제목", "기사내용"])
        writer.writeheader()
        writer.writerows(all_articles)

    print(f"총 {len(all_articles)}개의 무기체계 기사를 저장했습니다.")

# ✅ 사용 예시
base_url = "https://www.gukbangnews.com/news/articleList.html"
output_csv = "국방신문_무기체계.csv"
crawl_weapon_articles(base_url, output_csv)

# 드라이버 종료
driver.quit()


크롤링 중: 1 페이지
크롤링 중: 2 페이지
크롤링 중: 3 페이지
크롤링 중: 4 페이지
크롤링 중: 5 페이지
크롤링 중: 6 페이지
크롤링 중: 7 페이지
크롤링 중: 8 페이지
크롤링 중: 9 페이지
크롤링 중: 10 페이지
크롤링 중: 11 페이지
크롤링 중: 12 페이지
크롤링 중: 13 페이지
크롤링 중: 14 페이지
크롤링 중: 15 페이지
크롤링 중: 16 페이지
크롤링 중: 17 페이지
크롤링 중: 18 페이지
크롤링 중: 19 페이지
크롤링 중: 20 페이지
총 420개의 무기체계 기사를 저장했습니다.
